In [1]:
import pickle
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from proglearn.forest import UncertaintyForest
import time
from collections import Counter

In [2]:
(X, Y), (X_test_org, Y_test_org) = pickle.load(open('/home/weiwya/cifar_100_Bit_m-r101x1_embd.p', 'rb'))
print(X.shape, X_test_org.shape)

clusters = pickle.load(open('/home/weiwya/cifar_tenth_data_clusters.pkl', 'rb'))


(50000, 2048) (10000, 2048)


In [10]:
def gen_fine_level_data(X, fine_label, coarse_label,  X_test,  fine_label_test, coarse_label_test, wanted_idx):
    selected = np.where(coarse_label==wanted_idx)[0]
    X_selected = X[selected]
    Y_fine_selected = fine_label[selected]
    le = LabelEncoder().fit(Y_fine_selected)
    Y_fine_selected = le.transform(Y_fine_selected)
    
    selected_test = np.where(coarse_label_test == wanted_idx)[0]
    X_test_selected = X_test[selected_test]
    Y_fine_test_selected = fine_label_test[selected_test]
    Y_fine_test_selected = le.transform(Y_fine_test_selected)
    return X_selected, Y_fine_selected, X_test_selected, Y_fine_test_selected, le


def gen_fine_clf (X, Y_coarse, Y_fine, X_test, Y_coarse_test, Y_fine_test, wanted_idx, n_estimators=100, max_depth=10):
    X_fine_train, Y_fine_train, X_fine_test, Y_fine_test, le = gen_fine_level_data(X, Y_fine, Y_coarse, X_test, Y_fine_test, Y_coarse_test, wanted_idx )
    
    print(X_fine_train.shape, X_fine_test.shape)
    clf_fine = UncertaintyForest(n_estimators=n_estimators,
                        max_depth=max_depth)
    clf_fine.fit(X_fine_train, Y_fine_train)
    
#     pp = clf_fine.predict(X_fine_test, task_id=0)    
#     print(classification_report( Y_fine_test, pp))
    
    return clf_fine, le

def eval_hieracy_cluster_one_step(data, coarse_clf, fine_clfs):
    corse_level = coarse_clf.predict(data)
    res = []
    for i, c in enumerate(corse_level):
        fine_clf, le = fine_clfs[c]
        p = fine_clf.predict(data[i].reshape(1,-1))
        res.append(le.inverse_transform(p)[0])

    return res

def eval_hieracy_cluster_two_step(data, coarse_clf, fine_clfs):
    #get Corase level probality
    print('done coarse %s ' %(data.shape[0]))
    coarse_prob = coarse_clf.predict_proba(data)
    fine_probs, fine_res = [],[]
    
    for idx, (fine_clf, le) in fine_clfs.items():
        fine_prob = fine_clf.predict_proba(data)
        classes = fine_prob.argmax(axis=1)
        print(classes.shape)
        classes = le.inverse_transform(classes)
        #baysian
        fine_prob = coarse_prob[:, idx].reshape(-1,1) * fine_prob
        
        fine_prob = np.max(fine_prob, axis=1)
        fine_probs.append(fine_prob)
        fine_res.append(classes)
        print('done fine %s' %idx)

        
    fine_probs  = np.array(fine_probs)
    wanted  = fine_probs.argmax(axis=0)
    res = []
    for idx, w in enumerate(wanted):
        res.append(fine_res[w][idx])
    return res

def hieracy_cluster(X_train, Y_train, X_test, Y_test, used_hierarchy, n_estimators=100, max_depth=10 ):
    #relabel classes to hierarchy labels
    hierarchy_relabel = {i:used_hierarchy[i] for i in range(used_hierarchy.shape[0])    }
    
    #change fine label to corse label
    Y_train_hierarchy = np.array([hierarchy_relabel[x[0]] for x in Y_train])
    Y_test_hierarchy = np.array([hierarchy_relabel[x[0]] for x in Y_test])
    
    clf_coarse = UncertaintyForest(n_estimators=n_estimators,
                        max_depth=max_depth)
    clf_coarse.fit(X_train, Y_train_hierarchy)
    
    print('done train corase labels')
    
    fine_clfs= {}
    for wanted_idx in np.unique(used_hierarchy):
        print('processing %s' %wanted_idx)
        fine_clf, le = gen_fine_clf(X_train, Y_train_hierarchy, Y_train, X_test, Y_test_hierarchy, 
                                    Y_test, wanted_idx, n_estimators=n_estimators, max_depth=max_depth )
        fine_clfs[wanted_idx] = (fine_clf, le)
    
    return clf_coarse, fine_clfs

In [12]:
#just use 1st cluster generate by hayen
used_clusters = clusters[0]
n_iter = 10
n_trees = 100
max_depth =10
clfs = []
accuracies_1step = []
accuracies_2step = []

master_seed = 42
np.random.seed(master_seed)
seeds = np.random.randint(10000, size=n_iter)




for iteration in range(n_iter):
    start = time.time()
    seed =  seeds[iteration]
    print(iteration)
    X_train, _, Y_train, _ = train_test_split(X, Y, test_size=0.9, random_state=seed)
    print(X_train.shape, X_test_org.shape)
        
    clf_coarse, clf_fines = hieracy_cluster(X_train, Y_train, X_test_org, Y_test_org, used_clusters, n_estimators=n_trees, 
                                            max_depth=max_depth)
    
    
    
    clfs.append((clf_coarse, clf_fines))
    pp = eval_hieracy_cluster_one_step(X_test_org, clf_coarse, clf_fines)
    
    accuracies_1step.append(accuracy_score(Y_test_org, pp))
    
    pp = eval_hieracy_cluster_two_step(X_test_org, clf_coarse, clf_fines)
    accuracies_2step.append(accuracy_score(Y_test_org, pp))

    
    print(accuracies_1step[-1], accuracies_2step[-1])
    print('took %s' %(time.time() - start))
    print()

with open('/home/weiwya/cirar_hayden_10_10_run.p', 'wb') as handle:
    pickle.dump((clfs, accuracies_1step, accuracies_2step), handle)

print(np.average(accuracies_1step), np.average(accuracies_2step))
 
    
    
        

0
(5000, 2048) (10000, 2048)
done train corase labels
processing 0
(207, 2048) (400, 2048)


/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


processing 1
(261, 2048) (500, 2048)


/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


processing 2
(227, 2048) (500, 2048)


...

processing 25
(98, 2048) (200, 2048)


/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


processing 26
(50, 2048) (100, 2048)


/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


done coarse 10000 
(10000,)
done fine 0
(10000,)
done fine 1
(10000,)
done fine 2
(10000,)
done fine 3
(10000,)
done fine 4
(10000,)
done fine 5
(10000,)
done fine 6
(10000,)
done fine 7
(10000,)
done fine 8
(10000,)
done fine 9
(10000,)
done fine 10
(10000,)
done fine 11
(10000,)
done fine 12
(10000,)
done fine 13
(10000,)
done fine 14
(10000,)
done fine 15
(10000,)
done fine 16
(10000,)
done fine 17
(10000,)
done fine 18
(10000,)
done fine 19
(10000,)
done fine 20
(10000,)
done fine 21
(10000,)
done fine 22
(10000,)
done fine 23
(10000,)
done fine 24
(10000,)
done fine 25
(10000,)
done fine 26
0.4558 0.5389
took 1451.5436255931854

0.45845 0.5378000000000001


In [22]:
condition_cluster = pickle.load(open('/home/weiwya/cifar_cond_means_clusters.pkl', 'rb'))
#just use 1st cluster generate by hayen
used_clusters = condition_cluster[0]
n_iter = 10
n_trees = 100
max_depth =10
clfs_cond = []
accuracies_1step_cond = []
accuracies_2step_cond = []

master_seed = 42
np.random.seed(master_seed)
seeds = np.random.randint(10000, size=n_iter)




for iteration in range(n_iter):
    start = time.time()
    seed =  seeds[iteration]
    print(iteration)
    X_train, _, Y_train, _ = train_test_split(X, Y, test_size=0.9, random_state=seed)
    print(X_train.shape, X_test_org.shape)
        
    clf_coarse, clf_fines = hieracy_cluster(X_train, Y_train, X_test_org, Y_test_org, used_clusters, n_estimators=n_trees, 
                                            max_depth=max_depth)
    
    
    
    clfs_cond.append((clf_coarse, clf_fines))
    pp = eval_hieracy_cluster_one_step(X_test_org, clf_coarse, clf_fines)
    
    accuracies_1step_cond.append(accuracy_score(Y_test_org, pp))
    
    pp = eval_hieracy_cluster_two_step(X_test_org, clf_coarse, clf_fines)
    accuracies_2step_cond.append(accuracy_score(Y_test_org, pp))

    
    print(accuracies_1step_cond[-1], accuracies_2step_cond[-1])
    print('took %s' %(time.time() - start))
    print()

with open('/home/weiwya/cirar_hayden_conditional_10_10_run.p', 'wb') as handle:
    pickle.dump((clfs_cond, accuracies_1step_cond, accuracies_2step_cond), handle)

print(np.average(accuracies_1step_cond), np.average(accuracies_2step_cond))
 

0
(5000, 2048) (10000, 2048)
done train corase labels
processing 0
(257, 2048) (500, 2048)


/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


processing 1
(93, 2048) (200, 2048)


/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


processing 2
(532, 2048) (1000, 2048)


...

processing 26
(50, 2048) (100, 2048)
processing 27
(45, 2048) (100, 2048)


/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


processing 28
(51, 2048) (100, 2048)
processing 29
(47, 2048) (100, 2048)


/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


done coarse 10000 
(10000,)
done fine 0
(10000,)
done fine 1
(10000,)
done fine 2
(10000,)
done fine 3
(10000,)
done fine 4
(10000,)
done fine 5
(10000,)
done fine 6
(10000,)
done fine 7
(10000,)
done fine 8
(10000,)
done fine 9
(10000,)
done fine 10
(10000,)
done fine 11
(10000,)
done fine 12
(10000,)
done fine 13
(10000,)
done fine 14
(10000,)
done fine 15
(10000,)
done fine 16
(10000,)
done fine 17
(10000,)
done fine 18
(10000,)
done fine 19
(10000,)
done fine 20
(10000,)
done fine 21
(10000,)
done fine 22
(10000,)
done fine 23
(10000,)
done fine 24
(10000,)
done fine 25
(10000,)
done fine 26
(10000,)
done fine 27
(10000,)
done fine 28
(10000,)
done fine 29
0.451 0.4983
took 1236.0466248989105

0.45294999999999996 0.5104900000000001


In [25]:
print("hadyen average:%s std_err: %s " %(np.average(accuracies_2step), np.std(accuracies_2step)/np.sqrt(n_iter)   ))
print("hadyen_conditional average:%s std_err: %s " %(np.average(accuracies_2step_cond), np.std(accuracies_2step_cond)/np.sqrt(n_iter)   ))

hadyen average:0.5378000000000001 std_err: 0.0029206848511950058 
hadyen_conditional average:0.5104900000000001 std_err: 0.004470625235914991 
